In [ ]:
!cp -r ../input/updated-code-3-7/* /kaggle/working/

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install tqdm optuna ipykernel matplotlib lightfm

In [ ]:
!python run_compile_all_cython.py

In [ ]:
import numpy as np
import matplotlib.pyplot as pyplot
import pandas as pd
import scipy.sparse as sps
import matplotlib.pyplot as plt
from scipy.stats import loguniform
import time
import optuna

In [ ]:
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender
from Recommenders.NonPersonalizedRecommender import TopPop

#---- CF
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

#---- Matrix Factorization
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#---- CF w/ ML
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.Recommender_import_list import *

#---- Others
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Recommender_utils import check_matrix


In [ ]:
seed = 69
np.random.seed(seed)

In [ ]:
# Import training data
URM_path = "../input/data-books/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path,
                                header=0,
                                dtype={0:int, 1:int, 2:int},
                                engine='python')

URM_all_dataframe.columns = ["user_id", "item_id", "interaction"]

In [ ]:
# Import target users
target_path = "../input/data-books/data_target_users_test.csv"
target_dataframe= pd.read_csv(filepath_or_buffer=target_path,
                                header=0,
                                dtype={0:int},
                                engine='python')
target_dataframe.columns = ["user_id"]
target_dataframe

In [ ]:
def preprocess_data(ratings: pd.DataFrame):
    unique_users = ratings.user_id.unique()
    unique_items = ratings.item_id.unique()

    num_users, min_user_id, max_user_id = unique_users.size, unique_users.min(), unique_users.max()
    num_items, min_item_id, max_item_id = unique_items.size, unique_items.min(), unique_items.max()

    print(num_users, min_user_id, max_user_id)
    print(num_items, min_item_id, max_item_id)

    mapping_user_id = pd.DataFrame({"mapped_user_id": np.arange(num_users), "user_id": unique_users})
    mapping_item_id = pd.DataFrame({"mapped_item_id": np.arange(num_items), "item_id": unique_items})

    ratings = pd.merge(left=ratings,
                       right=mapping_user_id,
                       how="inner",
                       on="user_id")

    ratings = pd.merge(left=ratings,
                       right=mapping_item_id,
                       how="inner",
                       on="item_id")

    return ratings

In [ ]:
# Call preprocess data function
ratings = preprocess_data(URM_all_dataframe)

In [ ]:
URM = sps.coo_matrix((ratings.interaction.values, (ratings.mapped_user_id.values, ratings.mapped_item_id.values)))

In [ ]:
urm_train, urm_test = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
urm_train, urm_validation = split_train_in_two_percentage_global_sample(urm_train, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(urm_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(urm_test, cutoff_list=[10])

In [ ]:
class SaveResults(object):

    def __init__(self):
        self.results_df = pd.DataFrame(columns=["result"])

    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]

        # Create a DataFrame from the current trial's results
        trial_df = pd.DataFrame([hyperparam_dict])

        # Use concat instead of append
        self.results_df = pd.concat([self.results_df, trial_df], ignore_index=True)

In [ ]:
def objective_function_SLIM(optuna_trial):
    lambda_i = optuna_trial.suggest_float("lambda_i", 0, 1e-4)
    lambda_j = optuna_trial.suggest_float("lambda_j", 0, 1e-4)
    topK = optuna_trial.suggest_int("topK", 5, 1000)
    sgd_mode = optuna_trial.suggest_categorical("sgd_mode", ['adagrad', 'rmsprop', 'adam', 'sgd'])
    learning_rate = 1e-4
    random_seed = seed
    #gamma = optuna_trial.suggest_float("", 0.9, 1) - Use defaults
    #beta_1 = optuna_trial.suggest_float("", 0.89, 1)
    #beta_2 = optuna_trial.suggest_float("", 0.9, 1)

    recommender_instance = SLIM_BPR_Cython(urm_train, free_mem_threshold=0.1, verbose=False)
    recommender_instance.fit(
                             lambda_i = lambda_i,
                             lambda_j = lambda_j,
                             topK = topK,
                             sgd_mode = sgd_mode,
                             learning_rate = learning_rate,
                             random_seed = random_seed
                            )

    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)

    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study_SLIM = optuna.create_study(direction="maximize")

save_results_SLIM = SaveResults()

optuna_study_SLIM.optimize(objective_function_SLIM,
                      callbacks=[save_results_SLIM],
                      n_trials = 1000)

In [ ]:
pruned_trials = [t for t in optuna_study_SLIM.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in optuna_study_SLIM.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(optuna_study_SLIM.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
print("  Value Validation: ", optuna_study_SLIM.best_trial.value)

print("Best params:")
print(optuna_study_SLIM.best_trial.params)